
## Trial to combine middle ear and bruxism tagging to identify pure moments of middle ear activation

In [7]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import mne
from tinnsleep.config import Config
from tinnsleep.create_reports import preprocess, reporting
from tinnsleep.data import CreateRaw, RawToEpochs_sliding, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.classification import AmplitudeThresholding
from tinnsleep.check_impedance import create_annotation_mne, Impedance_thresholding_sliding, check_RMS, fuse_with_classif_result
from tinnsleep.signal import rms
from tinnsleep.scoring import classif_to_burst, burst_to_episode, create_list_events

import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import pandas as pd
import pickle
import scipy
from tinnsleep.config import Config

print("Config loaded")


Config loaded


In [2]:
#List of MEMA files, hardcoded, to be modified
print(Config.bruxisme_files[44:46])
print("")
ME_files=[Config.bruxisme_files[5], Config.bruxisme_files[9], 
          Config.bruxisme_files[14], Config.bruxisme_files[22]]
ME_files.extend(Config.bruxisme_files[28:44])
ME_files.extend(Config.bruxisme_files[46:])

print(ME_files)
print(len(ME_files))

['E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_23_sept.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_son_24_sept.edf']

['E:/Acou_sommeil/EDF_V2_PAUL\\1DA15_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MF19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1RA17_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_1_dec_OD__0to0.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_4_dec_OD__3to3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_5_dec_OD__0to1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Schmidtlin_nuit_2_dec_3to0to4.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Unger_2.edf', 'E:/Acou_somm

## Setting down parameters for bruxism detection

In [6]:
#Setting parameters
EDF_list = Config.bruxisme_files
os.chdir("C:/Users/Zeta/Documents/acou_sommeil_HD_ENS/Tinnitus-n-Sleep/Notebooks")
THR_classif=[[0,2]]
sfreq = 200
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping
n_adaptive = 480 # number of epochs for adaptative baseline

#Importing personnalized parameters for dataset
df = pd.read_pickle("data/valid_chans_THR_imp.pk")
dico_chans= df.to_dict("list")
print("parameters set")

parameters set


## Bruxism + MEMA processing for pure MEMA visualisation

In [9]:
filenames = ME_files  # load file from config
#filenames=['E:/Acou_sommeil/EDF_V2_PAUL\\sophie_mema.edf']

import warnings

labels_subj={}
ME_reports={}
#Setting parameters
EDF_list = Config.bruxisme_files
os.chdir("C:/Users/Zeta/Documents/acou_sommeil_HD_ENS/Tinnitus-n-Sleep/Notebooks")
THR_classif=[[0,2]]
sfreq = 200
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping
n_adaptive = 480 # number of epochs for adaptative baseline

#Importing personnalized parameters for dataset
df = pd.read_pickle("data/valid_chans_THR_imp.pickle")
dico_chans= df.to_dict("list")
print(dico_chans)
print("parameters set")

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    results={}
    print("Files processed : ")
    for filename in filenames:
        #opens the raw file
        raw = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
        print(filename.split("\\")[-1])
        #Get channels indexes
        ind_picks_chan= dico_chans[filename.split("\\")[-1]][0]
        ind_picks_imp= dico_chans[filename.split("\\")[-1]][1]
        #Get THR_imp value for filename
        THR_imp = dico_chans[filename.split("\\")[-1]][2]
        #Get channel names from indexes
        if len(ind_picks_chan)>0: #ignore file if no channel is good
            picks_chan=[]
            for elm in ind_picks_chan:
                picks_chan.append(raw.info["ch_names"][elm])
            picks_imp=[]
            for elm in ind_picks_imp:
                picks_imp.append(raw.info["ch_names"][elm])
            print(picks_chan)
            #Setting parameters for is_good
            params = dict(ch_names=picks_chan,
                  rejection_thresholds=dict(emg=1e-04),  # two order of magnitude higher q0.01
                  flat_thresholds=dict(emg=1e-09),  # one order of magnitude lower median
                  channel_type_idx=dict(emg=[ i for i in range(len(picks_chan))]),
                  full_report=True
                  )
            # Get the preprocessing steps done
            epochs, valid_labels, log = preprocess(raw, picks_chan, picks_imp, duration, interval, params, THR_imp=THR_imp, get_log=True)
            #If at least one epoch is good create report
            if np.sum(valid_labels)>0 : 
                results[filename] = reporting(epochs, valid_labels, THR_classif, n_adaptive, log)
                labels_subj[filename]={"bruxism" : results[filename]["labels"][0], "artefacts":valid_labels}
            else:
                labels_subj[filename]={"bruxism" : np.invert(valid_labels), "artefacts": valid_labels}
    print("Reports created, process finished.")


    for filename in filenames:
        if (labels_subj.keys().__contains__(filename)):
            print(filename)
            picks_chan = ['Airflow']           # middle ear electrodes

            raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
            raw  = CreateRaw(raw[picks_chan][0], picks_chan, ch_types=['emg'])        # pick channels and load

            ch_names = raw.info["ch_names"]
            print("Data filtered")

            #epoching
            sfreq = raw.info["sfreq"]
            window_length = 1                    # in seconds
            duration = int(window_length * sfreq)   # in samples
            interval = duration                     # no overlapping
            epochs = RawToEpochs_sliding(raw, duration=duration, interval=interval)
            print(f"Epochs done, shape {epochs.shape}")


            #Foward
            # compute the sum of power over electrodes and samples in each window
            pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=4, n_adaptive=60)
            X        = rms(epochs) # take only valid labels
            labels_f   = pipeline.fit_predict(X)


            #Backward
            #Reversing epochs array
            epochs = epochs[::-1]
             # compute the sum of power over electrodes and samples in each window
            pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=4, n_adaptive=60)
            X        = rms(epochs) # take only valid labels
            labels   = pipeline.fit_predict(X)
            #Reversing labels
            labels_b = labels[::-1]

            # Logical OR -- merged back and foward
            labels_fb = np.any(np.c_[labels_f, labels_b], axis=-1)
            labels_fb_shep=[]

            #adaptation of labels_fb fromù 1s epochs to 0,25s epochs
            for elm in labels_fb:
                for i in range(4):
                    labels_fb_shep.append(elm)

            #Creation of the list of episodes from the labels list:
            burst_list = classif_to_burst(labels_subj[filename]['bruxism'], time_interval=0.25)
            li_ep = burst_to_episode(burst_list, delim=3)
            event_list = create_list_events(li_ep, 0.25, len(labels_subj[filename]['bruxism']) * 0.25)

            #Careful, len(labels_fb_shep) and len(bruxism) maybe different by 1, 2 or 3 due to rounding effects.
            print(len(labels_fb))
            print(len(labels_fb_shep))
            print(len(labels_subj[filename]['bruxism']))
            print(len(event_list))

            #grossir les événements de 4 valeurs de chaque côté:
            if event_list[0]==0:
                flag = False
            else:
                flag = True
            for i in range(len(event_list)-4):
                if not flag :
                    if not event_list[i+4] == 0:
                        for j in range(4):
                            event_list[i+j]=4
                        flag = True
                if flag : 
                    if event_list[i+1] == 0:
                        for j in range(4):
                            event_list[i+j+1]=4
                        flag = False
                        
            #grossir les événements de 2 valeurs de chaque côté pour labels_subj[filename]['bruxism']:
            if labels_subj[filename]['bruxism'][0]==0:
                flag = False
            else:
                flag = True
            for i in range(len(labels_subj[filename]['bruxism'])-2):
                if not flag :
                    if not labels_subj[filename]['bruxism'][i+2] == 0:
                        for j in range(2):
                            labels_subj[filename]['bruxism'][i+j]=True
                        flag = True
                if flag : 
                    if labels_subj[filename]['bruxism'][i+1] == 0:
                        for j in range(2):
                            labels_subj[filename]['bruxism'][i+j+1]=True
                        flag = False

            for i in range(len(labels_subj[filename]['bruxism'])):
                if i < len(labels_fb_shep) - 1 : #len(labels_fb_shep) and len(bruxism) may be different by 1, 2 or 3
                    if labels_subj[filename]['bruxism'][i]:
                        labels_fb_shep[i] = False
                    if not labels_subj[filename]['artefacts'][i]:
                        labels_fb_shep[i] = False
                    if not event_list[i] == 0:
                        labels_fb_shep[i] = False

            #Merge the labels together and multiply by 4 the numbers of inputs
            #merge and compare with bruxism and artefacts

            OM_burst = classif_to_burst(labels_fb_shep, time_interval=0.25)
            OM_ep= burst_to_episode(OM_burst, delim=1.5, min_burst_joining=0)
            li_OM = create_list_events(OM_ep, 0.25, 0.25* len(labels_subj[filename]['bruxism']))

            #All episodes as tonic
            for elm in li_OM:
                if elm!=0:
                    elm=True
                else:
                    elm = False


            print(len(li_OM))
            print(len(labels_fb_shep))
            miny = min(len(li_OM), len(labels_fb_shep))
            MEMA = np.any(np.c_[li_OM[:miny], labels_fb_shep[:miny]], axis=-1)



            print("number of middle ear events")
            print(np.sum(MEMA))
            #Adding annotations



            picks_chan = ['Airflow', '1', '2']           # subset of EMG electrodes

            raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
            raw  = CreateRaw(raw[picks_chan][0], picks_chan, ch_types='emg')        # pick channels and load
            raw  = raw.load_data()
            dat=raw.get_data()
            dat[1]=[dat[1][i]*1/(0.0005) for i in range(len(dat[1]))]
            dat[2]=[dat[2][i]*1/(0.0005) for i in range(len(dat[2]))]
            raw  = CreateRaw(dat, picks_chan, ch_types='emg') 

            raw  = raw.filter(20., 99., n_jobs=4, 
                              fir_design='firwin', filter_length='auto', phase='zero-double',
                              picks=['1', '2'])

            raw = CleanAnnotations(raw)

            dict_annotations = {1: "tot"}
            raw = AnnotateRaw_sliding(raw, MEMA, 
                            dict_annotations=dict_annotations, duration=50, interval=50)

            print("Raw annotated")


            #scalings = dict(emg=1)
            raw.plot(scalings = "auto")
            plt.title(filename)
            ME_reports[filename] = [MEMA, len(OM_ep), np.sum(MEMA)]

{'1BA07_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1BA07_nuit_son.edf': [[0, 4], [1, 5], 8000], '1CC05_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1CC05_nuit_son.edf': [[0, 6], [1, 7], 8000], '1DA15_nuit_son.edf': [[0, 4], [1, 5], 8000], '1DL12_nuit_hab.edf': [[0, 6], [1, 7], 8000], '1DL12_nuit_son.edf': [[0, 4], [1, 5], 8000], '1GB18_nuit_son.edf': [[0, 4], [1, 5], 8000], '1GB19_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1GF14_nuit_son.edf': [[0, 4], [1, 5], 8000], '1MA16_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1MA16_nuit_son.edf': [[0, 4], [1, 5], 8000], '1MF19_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1MF19_nuit_son.edf': [[0, 6], [1, 7], 8000], '1MN09_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1MN09_nuit_son.edf': [[0, 4], [1, 5], 8000], '1PI07_nuit_son.edf': [[0, 4], [1, 5], 8000], '1PT06_nuit_hab.edf': [[0, 6], [1, 7], 8000], '1PT06_nuit_son.edf': [[0, 6], [1, 7], 8000], '1RA17_nuit_son.edf': [[0, 4], [1, 5], 8000], '1SA14_nuit_hab.edf': [[0, 4], [1, 5], 8000], '1SA14_nuit_son.edf': [[0, 4], [1

In [11]:
for elm in ME_reports.keys():
    print(elm)
    print(ME_reports[elm][1])

E:/Acou_sommeil/EDF_V2_PAUL\1DA15_nuit_son.edf
15
E:/Acou_sommeil/EDF_V2_PAUL\1GB19_nuit_hab.edf
141
E:/Acou_sommeil/EDF_V2_PAUL\1MF19_nuit_hab.edf
37
E:/Acou_sommeil/EDF_V2_PAUL\HZB_nuit_1.edf
70
E:/Acou_sommeil/EDF_V2_PAUL\HZB_nuit_2.edf
49
E:/Acou_sommeil/EDF_V2_PAUL\HZB_nuit_3.edf
22
E:/Acou_sommeil/EDF_V2_PAUL\SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf
142
E:/Acou_sommeil/EDF_V2_PAUL\SCHMIDTLIN_nuit_4_dec_OD__3to3.edf
110
E:/Acou_sommeil/EDF_V2_PAUL\SCHMIDTLIN_nuit_5_dec_OD__0to1.edf
101
E:/Acou_sommeil/EDF_V2_PAUL\SCHM_nuit_1.edf
6
E:/Acou_sommeil/EDF_V2_PAUL\SCHM_nuit_2.edf
87
E:/Acou_sommeil/EDF_V2_PAUL\SCHM_nuit_3.edf
147
E:/Acou_sommeil/EDF_V2_PAUL\Schmidtlin_nuit_2_dec_3to0to4.edf
0
E:/Acou_sommeil/EDF_V2_PAUL\Unger_2.edf
272
E:/Acou_sommeil/EDF_V2_PAUL\jon_mema.edf
182
E:/Acou_sommeil/EDF_V2_PAUL\jose_mema.edf
29
E:/Acou_sommeil/EDF_V2_PAUL\robin_mema_nuit_1.edf
78
E:/Acou_sommeil/EDF_V2_PAUL\robin_mema_nuit_2.edf
166
E:/Acou_sommeil/EDF_V2_PAUL\sophie_mema.edf
130
E:/Acou_somme

## Small function to check if all channel selections are good

In [5]:
import warnings

labels_subj={}
EDF_list = Config.bruxisme_files

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    results={}
    print("Files processed : ")
    for filename in EDF_list:
        #opens the raw file
        raw = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
        print(filename.split("\\")[-1])
        #Get channels indexes
        ind_picks_chan= dico_chans[filename.split("\\")[-1]][0]
        ind_picks_imp= dico_chans[filename.split("\\")[-1]][1]
        #Get THR_imp value for filename
        THR_imp = dico_chans[filename.split("\\")[-1]][2]
        #print(raw.info["ch_names"])
        #Get channel names from indexes
        if len(ind_picks_chan)>0: #ignore file if no channel is good
            picks_chan=[]
            for elm in ind_picks_chan:
                picks_chan.append(raw.info["ch_names"][elm])
            picks_imp=[]
            for elm in ind_picks_imp:
                picks_imp.append(raw.info["ch_names"][elm])
            print(picks_chan)
            print(picks_imp)
            print("")

Files processed : 
1BA07_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1BA07_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1CC05_nuit_hab.edf
['1', '2']
['1 Impedance', '2 Impedance']

1CC05_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DA15_nuit_hab.edf
['1']
['1 Imp?dance']

1DA15_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DL12_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DL12_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GB18_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GB19_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GF14_nuit_hab.edf
1GF14_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MA16_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MA16_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MF19_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MF19_nuit_son.edf
['1', '2']
['1 Impedance', '2 Impedance']

1MN09_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp